In [1]:
import pandas as pd
from pyspark import RDD
from pyspark import SparkContext
from operator import add


In [12]:
genre_rdd=sc.pickleFile("../data/metadata-artist_terms/")
year_rdd=sc.pickleFile("../data/musicbrainz-songs/")
metadata_rdd=sc.pickleFile('../data/metadata-songs/')

f=genre_rdd.map(lambda x: (x[0],x[1]))

year=year_rdd.map(lambda x: (x[0],x[1]['year'][0]))

hotness=metadata_rdd.map(lambda x: (x[0],x[1]['song_hotttnesss'][0]))
joined=f.join(year).join(hotness)
s=joined.collect()

In [30]:
s[0][1][1]

0.81839437971637685

In [32]:
by=joined.map(lambda x: (x[1][0][1],x[1][0][0],x[1][1])) #year and feature
g=by.collect()

array(['grunge', 'alternative metal', 'heavy metal', 'hard rock',
       'alternative rock', 'rock', 'seattle', 'alternative pop rock',
       'soundtrack', 'metal', 'heavy', 'acoustic', 'lyrics', 'alternative',
       'classic', 'beautiful', 'soul', 'sludge', 'guitar', 'mellow',
       'epic', 'aggressive', 'high school'],
      dtype='|S256')

In [11]:
m.reduceByKey?